# Detector Graph Creator
A notebook to help create detector graphs.

In [1]:
import yaml

In [2]:
class DetectorInfo:
    def __init__(self, module: str, class_: str, config: str = None) -> None:
        self.module = module
        self.class_ = class_
        self.config = config

class DetectorGraphCreator:
    def __init__(self) -> None:
        self.adjacency_list: dict[str, list[str]] = {}
        self.init_info =  {}
    
    def add_detectors(self, parent: str, node: str | list[str], detector: DetectorInfo = None) -> None:
        if node == "start":
            raise KeyError(f"'start' is reserved for parent nodes only")
        if parent == "end":
            raise KeyError(f"'end' cannot be a parent node")

        if parent not in self.adjacency_list:
            self.adjacency_list[parent] = [parent]
    
        if isinstance(node, list):
            for n in node:
                if n in self.adjacency_list[parent]:
                    raise KeyError(f"node {n} is already in {parent}'s adjacency list")
                self.adjacency_list[parent].append(n)
                self.add_init_info(n, detector)
        else:
            if node in self.adjacency_list[parent]:
                raise KeyError(f"node {node} is already in {parent}'s adjacency list")
            self.adjacency_list[parent].append(node)
            self.add_init_info(node, detector)

    def add_starting_detector(self, node: str, detector: DetectorInfo = None) -> None:
        self.add_detectors("start", node, detector)

    def add_ending_detector(self, parent: str, detector: DetectorInfo = None) -> None:
        self.add_detectors(parent, "end", detector)

    def to_yaml(self, path: str) -> None:
        contents = {"adjacency_list": list(self.adjacency_list.values()), "init_info": self.init_info}
        with open(path, "w") as w:
            yaml.safe_dump(contents, w)

    def add_init_info(self, node: str, detector: DetectorInfo) -> None:
        if node in self.init_info or node == 'end':
            return
        elif detector is None:
            raise ValueError(f"first occurance of {node} requires detector info.")

        self.init_info[node] = {}
        self.init_info[node]["module"] = detector.module
        self.init_info[node]["class"] = detector.class_
        self.init_info[node]["config"] = detector.config

In [7]:
dgc = DetectorGraphCreator()
perfect_detector_info = DetectorInfo("example_detectors", "PerfectDetector")

dgc.add_starting_detector("d1", perfect_detector_info)
dgc.add_detectors("d1", ["d2", "d3"], perfect_detector_info)
dgc.add_detectors("d3", "d4", perfect_detector_info)
dgc.add_detectors("d4", "d2", perfect_detector_info)
dgc.add_ending_detector("d2", perfect_detector_info)

print(dgc.adjacency_list)
print(dgc.init_info)

dgc.to_yaml("/home/70d/cyberwheel/cyberwheel/resources/configs/detector_handler.yaml")

{'start': ['start', 'd1'], 'd1': ['d1', 'd3'], 'd3': ['d3', 'd4'], 'd4': ['d4', 'end']}
{'d1': {'module': 'example_detectors', 'class': 'PerfectDetector', 'config': None}, 'd3': {'module': 'example_detectors', 'class': 'PerfectDetector', 'config': None}, 'd4': {'module': 'example_detectors', 'class': 'PerfectDetector', 'config': None}}
